In [1]:
from CustomPongEnv_v0_ram import Custom_Pong

In [2]:
env = Custom_Pong()
env.observation_space.shape

(6,)

In [4]:
%%time

episodes = 500
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    
    while not done:
        env.render()
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
#         print(state.shape)
    #print('Episode:{} Score:{}'.format(episode, reward))
    
env.close()

CPU times: total: 2min 25s
Wall time: 29min 30s


In [ ]:
env.action_space.sample()

In [ ]:
height, width, color = env.observation_space.shape
actions = env.action_space.n
height, width, color, actions

In [ ]:
import numpy as np
import tensorflow, keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# del model

In [ ]:
def build_model(height, width, color, actions):
    model=Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape = (3,height, width, color)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(2, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

model = build_model(height, width, color, actions)
model.summary()

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))